In [2]:
import torch
import torch.nn as nn
import torchvision #for dataset 
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\tahah\anaconda3\envs\riemann\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] La procédure spécifiée est introuvable'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
input_size = 28*28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
lr = 0.01

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset= train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset= test_dataset, batch_size = batch_size, shuffle = True)

In [5]:
examples = iter(train_loader)
samples, labels = next(examples)

In [27]:
class PortfolioRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(PortfolioRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        # x -> (batch_size, timesteps/seq, input_size/feature_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, X, r):
        h0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(X,h0)
        # out -> (batch_size, seq_length, hidden_size)
        #out = out[:,-1,:] only take the last time step NO WE NEED ALL TIME STEPS
        out = self.linear(out)
        out = torch.softmax(out, dim=-1)
        portfolio_returns = torch.sum(out[:, :, :] * r[:, :, :], dim=-1)
        # portfolio_returns -> (batch_size)
        sharpe = torch.mean(portfolio_returns, dim = -1) / torch.std(portfolio_returns, dim = -1)
        return -sharpe.mean()

In [28]:
model = PortfolioRNN(5, 1, 1, 2).to(device)

# Generate random input data
# X -> (batch_size, seq_length, input_size)
X = torch.rand(2, 10, 5).to(device)

# r -> (batch_size, seq_length, output_size)
r = torch.rand(2, 10, 2).to(device)

# Call the forward method
loss = model(X, r)

# Print the output
print("Output loss (negative Sharpe ratio):", loss.item())

Output loss (negative Sharpe ratio): -2.727998733520508


In [20]:
import numpy as np
import yfinance as yf
import pandas as pd

tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2006-01-01", end="2020-12-31", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
for column in data_na.columns:
    data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
    data_na[f'{column}_y'] = data_na[f'{column}_R'].shift(-1)
data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date
data_na

[*********************100%***********************]  4 of 4 completed
C:\Users\tahah\AppData\Local\Temp\ipykernel_14048\121768413.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
C:\Users\tahah\AppData\Local\Temp\ipykernel_14048\121768413.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_na[f'{column}_R'] = data_na[f'{column}'].pct_change()
C:\Users\tahah\AppData\Local\Temp\ipykernel_14048\121768413.py:9: SettingWithCopyWarning: 
A value is

Ticker,Date,AGG,DBC,VTI,^VIX,AGG_R,DBC_R,VTI_R,^VIX_R
0,2006-02-07,56.438545,20.285255,44.219509,13.590000,-0.000699,-0.028926,-0.009736,0.042178
1,2006-02-08,56.410370,20.198933,44.537621,12.830000,-0.000499,-0.004255,0.007194,-0.055923
2,2006-02-09,56.444210,20.388840,44.452801,13.120000,0.000600,0.009402,-0.001904,0.022603
3,2006-02-10,56.325832,20.017662,44.544701,12.870000,-0.002097,-0.018205,0.002067,-0.019055
4,2006-02-13,56.365330,19.706909,44.343197,13.350000,0.000701,-0.015524,-0.004524,0.037296
...,...,...,...,...,...,...,...,...,...
3746,2020-12-23,106.686798,13.794738,182.168091,23.309999,-0.000678,0.012517,0.001711,-0.037969
3747,2020-12-24,106.786331,13.832635,182.472778,21.530001,0.000933,0.002747,0.001673,-0.076362
3748,2020-12-28,106.804459,13.747366,183.627289,21.700001,0.000170,-0.006164,0.006327,0.007896
3749,2020-12-29,106.822563,13.785263,182.860779,23.080000,0.000170,0.002757,-0.004174,0.063594


In [29]:
def create_batches(dataframe, start_date, end_date, window_size=50):
    dataframe['Date'] = pd.to_datetime(dataframe['Date'])
    
    filtered_data = dataframe[(dataframe['Date'] >= start_date) & (dataframe['Date'] <= end_date)]
    
    x_batches = []
    y_batches = []
    dates_batches = []

    for i in range(len(filtered_data) - window_size):
        window_returns = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col.endswith('_R')]].values
        window_prices = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col in ['AGG', 'DBC', 'VTI', '^VIX']]].values
        
        window_x = np.concatenate([window_returns, window_prices], axis=1)
        window_y = filtered_data.iloc[i:i+window_size][[col for col in dataframe.columns if col.endswith('_y')]].values
        
        window_dates = filtered_data.iloc[i:i+window_size]['Date'].values

        x_batches.append(window_x)
        y_batches.append(window_y)
        dates_batches.append(window_dates)

    return np.array(x_batches), np.array(y_batches), np.array(dates_batches)

x_batches, y_batches, dates_batches = create_batches(data_na, start_date="2006-01-01", end_date="2010-12-31")

C:\Users\tahah\AppData\Local\Temp\ipykernel_14048\3333825469.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['Date'] = pd.to_datetime(dataframe['Date'])


In [30]:
mini_batch_size = 64

x_batches_tensor = torch.tensor(x_batches, dtype=torch.float32)
y_batches_tensor = torch.tensor(y_batches, dtype=torch.float32)

num_batches = x_batches_tensor.shape[0]

input_size = 8
hidden_size = 64
output_size = 4
model_rnn = PortfolioRNN(input_size=input_size, hidden_size=hidden_size, num_layers=2, output_size=output_size)

In [34]:
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=0.001)
num_epochs = 100


for epoch in range(num_epochs):
    model_rnn.train()
    loss_epoch = 0
    
    for i in range(0, num_batches - mini_batch_size + 1, mini_batch_size):
        x_mini_batch = x_batches_tensor[i:i+mini_batch_size]
        y_mini_batch = y_batches_tensor[i:i+mini_batch_size]
        
        # Check the shapes
        print(f"Batch {i // mini_batch_size + 1}:")
        print(f"x_mini_batch shape: {x_mini_batch.shape}")
        print(f"y_mini_batch shape: {y_mini_batch.shape}")
        
        optimizer.zero_grad()
        
        loss = model_rnn(x_mini_batch, y_mini_batch)
        loss_epoch += loss.item()
        loss.backward()
        optimizer.step()
    
    print(f"epoch [{epoch+1}/{num_epochs}], loss: {(loss_epoch/mini_batch_size)}")

Batch 1:
x_mini_batch shape: torch.Size([64, 50, 8])
y_mini_batch shape: torch.Size([64, 50, 0])


RuntimeError: The size of tensor a (4) must match the size of tensor b (0) at non-singleton dimension 2

In [12]:
model = PortfolioRNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [13]:
sequence_length = 28
input_size = 28

for epoch in range(len(train_loader)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad
        loss.backward()
        optimizer.step()

        if (i+1)% 100 ==0:
            print(f'Epoch {epoch+1}/{num_epochs}, Step {i+1}/{len(train_loader)}, Loss = {loss.item()}')

Epoch 1/2, Step 100/600, Loss = 1.7561472654342651
Epoch 1/2, Step 200/600, Loss = 1.1913491487503052
Epoch 1/2, Step 300/600, Loss = 0.4377589523792267
Epoch 1/2, Step 400/600, Loss = 1.1186847686767578
Epoch 1/2, Step 500/600, Loss = 1.1297461986541748
Epoch 1/2, Step 600/600, Loss = 1.624846339225769
Epoch 2/2, Step 100/600, Loss = 0.710981011390686
Epoch 2/2, Step 200/600, Loss = 0.7087716460227966
Epoch 2/2, Step 300/600, Loss = 1.1653952598571777
Epoch 2/2, Step 400/600, Loss = 1.0352438688278198
Epoch 2/2, Step 500/600, Loss = 0.6385011076927185
Epoch 2/2, Step 600/600, Loss = 0.748671293258667
Epoch 3/2, Step 100/600, Loss = 1.1222331523895264
Epoch 3/2, Step 200/600, Loss = 0.8393617272377014
Epoch 3/2, Step 300/600, Loss = 0.7829921245574951
Epoch 3/2, Step 400/600, Loss = 0.897032618522644
Epoch 3/2, Step 500/600, Loss = 0.9496373534202576
Epoch 3/2, Step 600/600, Loss = 0.8489352464675903
Epoch 4/2, Step 100/600, Loss = 0.5647134780883789
Epoch 4/2, Step 200/600, Loss = 1.4

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 784).to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape(0)
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy = {acc}') 